In [76]:
!pip install PyPDF2
import os
import re
import pandas as pd
from PyPDF2 import PdfReader

folder_path = r'C:\Users\Priyank\Downloads\Client data'  # Replace with your folder path

file_list = os.listdir(folder_path)

result = []  # Variable to store the extracted data

for file_name in file_list:
    if file_name.endswith('.pdf'):  # Process only PDF files
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'rb') as file:
            pdf = PdfReader(file)
            num_pages = len(pdf.pages)
            text = ""
            for page_num in range(num_pages):
                page = pdf.pages[page_num]
                text += page.extract_text()
                
            partitions = re.split(r'Text From \w+', text)
            
            # Define the desired columns to extract
            desired_columns = ['Authorised capital of the company as on the date of filing','Corporate identity number (CIN) of company','e-mail id of the company']
            data_list = []
            # Extract the data from the text
            result = []
            for partition in partitions:
                partition_data = {}
                for column in desired_columns:
                    match = re.search(fr'{column}\n.*?([\d,]+)', partition)
                    match1 = re.search(fr"(\w+@\w+(?:\.\w+)+)[\n]*.*{re.escape(column)}", partition)
                    match2 = re.search(fr"([\w]+[\d]+)[\n]*.*{re.escape(column)}", partition)
                    if match:
                        partition_data[column] = match.group(1).strip()
                    elif match1:
                        partition_data[column] = match1.group(1).strip()
                    elif match2:
                        partition_data[column] = match2.group(1).strip()
                result.append(partition_data)
        data_list.extend(result)

# Create DataFrame from the extracted data
df = pd.DataFrame(data_list)

df

,Authorised capital of the company as on the date of filing,Corporate identity number (CIN) of company,e-mail id of the company
0,"100,000",U29248GJ2009PTC057637,officeclient1@rediffmail.com
